In [12]:
import pandas as pd
import numpy as np

from basic_synthesizer import BasicSynthesizer

dataset = pd.read_csv("../normalizing_flows/datasets/adult_subset.csv")

synthesizer = BasicSynthesizer(dataset)

In [13]:
synth_data = synthesizer.sample(len(dataset))

In [14]:
synth_data

,0,1,2,3,4,5
0,45.403748,244091.234927,11.382755,4777.854804,281.672006,46.180017
1,36.861230,176200.803578,9.763389,143.792807,28.336407,38.671773
2,47.434843,260233.067865,11.767781,5879.663308,341.905839,47.965200
3,59.211020,353822.542100,14.000137,12267.889414,691.138345,58.315593
4,35.571212,165948.565061,9.518846,-556.004027,-9.920191,37.537942
...,...,...,...,...,...,...
2995,39.763697,199267.741950,10.313596,1718.294844,114.411522,41.222828
2996,42.188940,218542.010592,10.773339,3033.917003,186.334152,43.354438
2997,22.015630,58217.527828,6.949175,-7909.504302,-411.922451,25.623583
2998,43.217089,226713.075213,10.968240,3591.657163,216.824775,44.258105


In [15]:
dataset.describe()

,0,1,2,3,4,5
count,3000.000000,3.000000e+03,3000.000000,3000.000000,3000.000000,3000.000000
mean,38.721333,1.909837e+05,10.116000,1152.843333,83.499333,40.306667
std,13.455483,1.069355e+05,2.550695,7299.199688,399.033747,11.826379
min,17.000000,1.882700e+04,1.000000,0.000000,0.000000,2.000000
25%,28.000000,1.157738e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.817685e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.396855e+05,13.000000,0.000000,0.000000,45.000000
max,90.000000,1.268339e+06,16.000000,99999.000000,3900.000000,99.000000


In [16]:
synth_data.describe()

,0,1,2,3,4,5
count,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,39.151848,194405.159209,10.197611,1386.384891,96.266618,40.685058
std,13.275760,105507.190088,2.516626,7201.705017,393.703894,11.668415
min,-4.884216,-155565.122338,1.849893,-22501.870784,-1209.660138,1.980600
25%,30.278871,123888.452602,8.515601,-3426.941284,-166.869021,32.886364
50%,39.049119,193588.733524,10.178137,1330.657344,93.220096,40.594766
75%,47.783313,263002.487195,11.833839,6068.698208,352.240027,48.271480
max,91.541953,610767.980934,20.128966,29806.459577,1649.939539,86.732103


In [17]:
from metrics.pmse import pmse_ratio
from metrics.sra import sra
from metrics.wasserstein import wasserstein_randomization
import numpy as np

pmse_ratio(dataset, synth_data)

1.288126716256028

In [18]:
wass = []
for i in range(50):
  wass.append(wasserstein_randomization(dataset, dataset, 100))

print(np.mean(wass))
print(np.std(wass))

3123.521137929315
291.2967417156186


In [19]:
wass = []
for i in range(50):
  wass.append(wasserstein_randomization(dataset, synth_data, 100))

print(np.mean(wass))
print(np.std(wass))

3215.1424462816644
402.0660284918705


In [20]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from metrics.sra import sra

#orig_data = pd.read_csv('datasets/adult_subset.csv')
#synth_data = pd.read_csv('synth_data/synth_adult.csv')
#synth_data = synth_data.replace(-np.Inf, np.nan).replace(np.inf, np.nan).drop(synth_data.columns[0], axis=1).dropna()
orig_data = dataset

columns_to_predict = orig_data.columns

orig_accuracies = []
synth_accuracies = []
for column in columns_to_predict:
  X = orig_data.loc[:, orig_data.columns != column]
  y = orig_data.loc[:, column]
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)
  fit = LinearRegression().fit(X_train, y_train)
  predicted = fit.predict(X_test)
  orig_accuracies.append(r2_score(y_test, predicted))
  X = synth_data.loc[:, synth_data.columns != column]
  y = synth_data.loc[:, column]
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)
  fit = LinearRegression().fit(X_train, y_train)
  predicted = fit.predict(X_test)
  synth_accuracies.append(r2_score(y_test, predicted))
    
print(sra(orig_accuracies, synth_accuracies))

1.0
